In [268]:
from matplotlib import pyplot as plt 
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
# change file_data to where did you put it! 
file_data = 'D:\\Python\\MachineLearning\\MachineLearning\\week04\\data\\glass.csv' 
glass_df = pd.read_csv(file_data) 

print(glass_df.info()) 
glass_df.drop(columns='Id_Number', inplace=True)
glass_types = glass_df['Type'].unique() 
print(glass_types) 
print(glass_df['Type'].value_counts()) 
X = glass_df[glass_df.columns[:-1]] 
y = glass_df['Type'] 
y = np.asarray(y) - 1
X = np.asarray(X)
C = len(np.unique(y)) + 1# number of classes (for c = 1, 2, 3, 4, 5, 6, 7)
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25, random_state=42)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id_Number  214 non-null    int64  
 1   RI         214 non-null    float64
 2   Na         214 non-null    float64
 3   Mg         214 non-null    float64
 4   Al         214 non-null    float64
 5   Si         214 non-null    float64
 6   K          214 non-null    float64
 7   Ca         214 non-null    float64
 8   Ba         214 non-null    float64
 9   Fe         214 non-null    float64
 10  Type       214 non-null    int64  
dtypes: float64(9), int64(2)
memory usage: 18.5 KB
None
[1 2 3 5 6 7]
Type
2    76
1    70
7    29
3    17
5    13
6     9
Name: count, dtype: int64


In [269]:
from scipy import sparse  
def convert_labels(y, C = C): 
    """ 
    convert 1d label to a matrix label: each column of this  
    matrix coresponding to 1 element in y. In i-th column of Y,  
    only one non-zeros element located in the y[i]-th position,  
    and = 1 ex: y = [0, 2, 1, 0], and 3 classes then return 
 
            [[1, 0, 0, 1], 
             [0, 0, 1, 0], 
             [0, 1, 0, 0]] 
    """ 
    Y = sparse.coo_matrix((np.ones_like(y), (y, np.arange(len(y)))), shape = (C, len(y))).toarray() 
    return Y  
 
# Y = convert_labels(y, C) 

In [270]:
def softmax_stable(Z): 
    """
     Compute softmax values for each sets of scores in Z. 
    each column of Z is a set of score.     
    """ 
    e_Z = np.exp(Z - np.max(Z, axis = 0, keepdims = True)) 
    A = e_Z / e_Z.sum(axis = 0) 
    return A 
def softmax(Z): 
    """ 
    #Compute softmax values for each sets of scores in V. 
    #each column of V is a set of score.     
    """ 
    e_Z = np.exp(Z) 
    A = e_Z / e_Z.sum(axis = 0) 
    return A 
def softmax_regression(X, y, W_init, eta, tol = 1e-4, max_count = 10000): 
    W = [W_init]     
    C = W_init.shape[1] 
    Y = convert_labels(y, C) 
    it = 0 
    N = X.shape[1] 
    d = X.shape[0] 
     
    count = 0 
    check_w_after = 20 
    while count < max_count: 
        # mix data  
        mix_id = np.random.permutation(N) 
        for i in mix_id: 
            xi = X[:, i].reshape(d, 1) 
            yi = Y[:, i].reshape(C, 1) 
            ai = softmax_stable(np.dot(W[-1].T, xi)) 
            W_new = W[-1] + eta*xi.dot((yi - ai).T) 
            count += 1 
            # stopping criteria 
            if count%check_w_after == 0:                 
                if np.linalg.norm(W_new - W[-check_w_after]) < tol: 
                    return W 
            W.append(W_new) 
    return W 
 
# cost or loss function   
def cost(X, Y, W): 
    A = softmax(W.T.dot(X)) 
    return -np.sum(Y*np.log(A)) 
 
# Predict that X belong to which class (1..C now indexed as 0..C-1 )  
def pred(W, X): 
    """ 
    predict output of each columns of X 
    Class of each x_i is determined by location of max probability 
    Note that class are indexed by [0, 1, 2, ...., C-1] 
    """ 
    A = softmax_stable(W.T.dot(X)) 
    return np.argmax(A, axis = 0) 
 
# W[-1] is the solution, W is all history of weights

In [271]:
eta = .05  
d = X.T.shape[0]
W_init = np.random.randn(X.T.shape[0], C) 
W = softmax_regression(X.T, y, W_init, eta) 
print(W[-1]) 

[[ 4.64294324e-01  1.20107946e-02  1.72252368e+00  4.13084221e-01
   3.75271199e-02 -4.86146953e-01 -1.83217333e-01]
 [-1.39576832e+01 -3.34027256e+01  7.40649440e+00  3.31678507e-01
  -1.17237435e+01  2.25876033e+01  3.30215685e+01]
 [ 7.05271655e+01  2.19410382e+01  2.18178938e+01  1.54826732e+00
  -3.62700915e+01 -1.85186700e+01 -6.37141328e+01]
 [-3.41133165e+01  3.69099040e+00 -7.35995581e+00 -1.37483598e+00
   1.68025092e+01 -4.30833115e+00  2.53664297e+01]
 [ 3.44454224e+00  1.57007271e+00 -1.64631035e+00 -4.97695617e-01
   2.87267043e-01 -2.26060718e+00 -2.15301427e+00]
 [-6.39393576e+00  5.53659565e+00 -3.15592871e+00 -6.97631909e-01
   3.01154876e+01 -9.07713318e+00 -9.24909563e+00]
 [ 1.75046996e+00  1.33904069e+01 -3.03260272e+00 -9.97733259e-01
   1.42307642e+01  5.98845028e-01 -2.57132869e+01]
 [-1.24607525e+01 -1.15247482e+01 -5.41656749e+00  6.45856757e-02
  -6.51707398e-01 -6.51707701e+00  3.14037855e+01]
 [-1.43671341e+00  5.04394425e+00  1.14927055e+00 -2.35651128e+0

In [272]:
import sklearn 
# Applying logistic sigmoid regression to find coefficients
w_init = np.random.randn(X_train.T.shape[0], C)
eta = 0.05
w = softmax_regression(X_train.T, y_train.T, w_init, eta)

# Print coefficients
print("Coefficients:")
print(w[-1])

Y_predict = pred(w[-1], X_test.T)
print(Y_predict)

# for accuracy 
from sklearn.metrics import accuracy_score 
print("Accuracy Score: ")
print(accuracy_score(y_test,Y_predict)) 
 
# for confusion matrix 
from sklearn.metrics import confusion_matrix 
cm=confusion_matrix(y_test,Y_predict) 
print("Confusion Matrix:")
print(cm) 

Coefficients:
[[-6.69916566e-03  8.82041324e-01  5.29977991e-02  8.33019900e-01
  -1.57328039e-01 -1.85765092e+00 -4.83550916e-03]
 [-1.08637570e+01 -3.00109242e+01  4.70206226e+00 -1.68414164e-01
  -1.38761719e+01  1.56396291e+01  3.56986900e+01]
 [ 6.96856460e+01  1.76457114e+01  2.60640350e+01  9.05514904e-01
  -3.35181549e+01 -1.73095622e+01 -6.34021720e+01]
 [-3.06139572e+01  4.31310726e+00 -6.93091291e+00  4.66218180e-01
   1.57614905e+01 -6.53887398e-01  2.17013611e+01]
 [ 5.38888273e-01  4.35578815e+00 -1.11505674e+00 -3.78372689e-01
   1.28486358e+00 -2.10185322e+00 -8.15843904e-01]
 [-7.83642747e+00  2.64558906e+00 -5.26172486e-01 -8.29099041e-01
   2.30741412e+01 -1.08309090e+01 -6.85346373e+00]
 [-4.40493758e+00  2.04055978e+01 -3.01880447e+00  2.18554207e+00
   1.48757697e+01  4.67814702e+00 -3.33342643e+01]
 [-1.32231970e+01 -9.62817928e+00 -5.93078282e+00 -9.93465560e-01
   1.81519705e+00 -3.55768584e+00  3.44688282e+01]
 [-2.22920194e+00  4.93679723e+00  1.99659530e-01 

Code bằng thư viện Scikit learn

In [275]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import sklearn 
#from sklearn.preprocessing import StandardScaler  
from sklearn.linear_model import LogisticRegression 

# Call to Logistic Regression Model - SAG: solving is based on Stochastic Average Gradient 
lorg=LogisticRegression(multi_class='multinomial',solver='sag', max_iter=5000) 
# and train model by Training Dataset 
lorg.fit(X_train,y_train)  
 
# Then Predict the Test data 
Y_pred=lorg.predict(X_test) 
 
# for accuracy 
from sklearn.metrics import accuracy_score 
print("Accuracy Score: ")
print(accuracy_score(y_test,Y_pred)) 
 
# for confusion matrix 
from sklearn.metrics import confusion_matrix 
cm=confusion_matrix(y_test,Y_pred) 
print("Confusion Matrix:")
print(cm) 
print("Coefficients:")
print(lorg.coef_)
y_predict = lorg.predict(X_test)
print("Predict:")
print(y_predict)

Accuracy Score: 
0.7037037037037037
Confusion Matrix:
[[12  2  0  0  0  0]
 [ 4 16  0  0  1  0]
 [ 2  2  0  0  0  0]
 [ 0  2  0  2  0  0]
 [ 0  1  0  0  0  2]
 [ 0  0  0  0  0  8]]
Coefficients:
[[-2.87487924e-02 -6.13111726e-01  1.20719919e+00 -1.84856027e+00
   9.62076558e-02 -2.93672752e-01  1.90300290e-01  1.58117744e-01
  -2.00034843e-01]
 [ 4.72709370e-02 -3.34480263e-01  5.25823844e-01  4.94128449e-01
  -1.81923594e-03  3.10803042e-01  4.27061281e-01  2.11775089e-01
   7.13063884e-01]
 [ 1.18549126e-02  1.79388352e-01  1.05268172e+00 -3.25097187e-01
  -1.13191709e-01 -2.12974647e-01  3.78947660e-01 -2.16235877e-01
  -9.90412204e-02]
 [ 2.11046342e-02 -6.30703698e-01 -6.26792161e-01  1.76844819e+00
   7.01361985e-02  9.28893691e-01  1.37464991e-01 -7.53715188e-02
  -1.36747669e-01]
 [-4.51379486e-02  8.01658078e-01 -7.06980035e-01 -3.57486289e-01
  -7.47868102e-02 -1.12592211e+00 -3.27870915e-01 -7.04066110e-01
  -1.69079183e-01]
 [-6.34374271e-03  5.97249257e-01 -1.45193256e+00 